<a href="https://colab.research.google.com/github/Yegeurina/HW_BigDataProject/blob/main/FuzzyAttack%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
import math

In [3]:
label=['TimeStamp','CAN ID','DLC','DATA0','DATA1','DATA2','DATA3','DATA4','DATA5','DATA6','DATA7','Flag']

In [4]:
fuzzy = pd.Cov = pd.read_csv('/content/drive/Shareddrives/빅응보 플젝/Fuzzy_dataset.csv',names=label)

In [5]:
fuzzy.astype('object')
fuzzy.astype({'TimeStamp' : 'float64'})
fuzzy.astype({'DLC':'int64'})
fuzzy.dtypes

TimeStamp    float64
CAN ID        object
DLC            int64
DATA0         object
DATA1         object
DATA2         object
DATA3         object
DATA4         object
DATA5         object
DATA6         object
DATA7         object
Flag          object
dtype: object

In [6]:
# 결측값 있는 행 제거
fuzzy.dropna(axis=0,how='any',inplace=True)

In [7]:
fuzzy['CAN ID']=fuzzy['CAN ID'].apply(lambda x : int(str(x),16)) 
for i in range(0,8):
  adr = 'DATA'+str(i)
  #print(adr)
  fuzzy[adr]=fuzzy[adr].apply(lambda x : int(str(x),16))

In [8]:
fuzzy['TimeStamp']=fuzzy['TimeStamp']*math.pow(10,9)
fuzzy['TimeStamp']=pd.to_datetime(fuzzy['TimeStamp'],format="%Y-%m-%d %H:%M:%S")

In [9]:
fuzzy['Diff']=fuzzy.TimeStamp.diff()
fuzzy

,TimeStamp,CAN ID,DLC,DATA0,DATA1,DATA2,DATA3,DATA4,DATA5,DATA6,DATA7,Flag,Diff
0,2016-11-03 17:55:21.903877120,1349,8,216,0,0,138,0,0,0,0,R,NaT
2,2016-11-03 17:55:21.908436992,2,8,0,0,0,0,0,1,7,21,R,0 days 00:00:00.004559872
3,2016-11-03 17:55:21.908675840,339,8,0,33,16,255,0,255,0,0,R,0 days 00:00:00.000238848
4,2016-11-03 17:55:21.909414144,304,8,25,128,0,255,254,127,7,96,R,0 days 00:00:00.000738304
5,2016-11-03 17:55:21.909659904,305,8,23,128,0,0,101,127,7,159,R,0 days 00:00:00.000245760
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3838855,2016-11-03 19:26:49.058007040,399,8,254,89,0,0,0,65,0,0,R,0 days 00:00:00.000225024
3838856,2016-11-03 19:26:49.058236928,608,8,24,33,33,48,8,143,109,25,R,0 days 00:00:00.000229888
3838857,2016-11-03 19:26:49.058471168,672,8,36,0,154,29,151,2,189,0,R,0 days 00:00:00.000234240
3838858,2016-11-03 19:26:49.058702848,809,8,220,183,127,20,17,32,0,20,R,0 days 00:00:00.000231680


In [10]:
fuzzy = fuzzy.sort_values(by=['CAN ID'],ascending=True)

In [11]:
grouped = fuzzy.groupby('CAN ID')
grouped.size()
Max_num = max(grouped.size())
Min_num = min(grouped.size())

In [12]:
def generate_lagged_features(fuzzy,var,max_lag) : 
  for t in range(1,max_lag+1) :
    fuzzy[var+'_lag'+str(t)]=fuzzy.groupby('CAN ID')[var].shift(t)

In [13]:
generate_lagged_features(fuzzy,'Diff',10) #차후 min max로 교환 예정
fuzzy

,TimeStamp,CAN ID,DLC,DATA0,DATA1,DATA2,DATA3,DATA4,DATA5,DATA6,DATA7,Flag,Diff,Diff_lag1,Diff_lag2,Diff_lag3,Diff_lag4,Diff_lag5,Diff_lag6,Diff_lag7,Diff_lag8,Diff_lag9,Diff_lag10
2494713,2016-11-03 18:29:23.247490048,0,8,59,28,72,233,183,245,37,3,T,0 days 00:00:00.005488128,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2138410,2016-11-03 18:23:25.855016960,0,8,159,128,141,72,22,174,22,254,T,0 days 00:00:00.005895936,0 days 00:00:00.005488128,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
424630,2016-11-03 18:01:08.118797056,0,8,211,24,136,179,33,119,14,36,T,0 days 00:00:00.000311040,0 days 00:00:00.005895936,0 days 00:00:00.005488128,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
722357,2016-11-03 18:04:25.968442880,0,8,169,50,163,67,69,13,210,239,T,0 days 00:00:00.000221696,0 days 00:00:00.000311040,0 days 00:00:00.005895936,0 days 00:00:00.005488128,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1469283,2016-11-03 18:12:41.194611968,0,8,200,20,207,212,252,4,149,219,T,0 days 00:00:00.005209088,0 days 00:00:00.000221696,0 days 00:00:00.000311040,0 days 00:00:00.005895936,0 days 00:00:00.005488128,NaT,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2042399,2016-11-03 18:21:51.213734912,2047,8,208,83,112,243,113,64,124,178,T,0 days 00:00:00.000303872,0 days 00:00:00.000709888,0 days 00:00:00.000854016,0 days 00:00:00.002191104,0 days 00:00:00.000310016,0 days 00:00:00.000302336,0 days 00:00:00.000301824,0 days 00:00:00.000222208,0 days 00:00:00.000571136,0 days 00:00:00.005938944,0 days 00:00:00.000218880
2885520,2016-11-03 18:35:49.679506944,2047,8,162,52,233,113,250,198,244,114,T,0 days 00:00:00.000221952,0 days 00:00:00.000303872,0 days 00:00:00.000709888,0 days 00:00:00.000854016,0 days 00:00:00.002191104,0 days 00:00:00.000310016,0 days 00:00:00.000302336,0 days 00:00:00.000301824,0 days 00:00:00.000222208,0 days 00:00:00.000571136,0 days 00:00:00.005938944
1153132,2016-11-03 18:09:14.729126912,2047,8,49,218,8,114,233,74,146,13,T,0 days 00:00:00.000221952,0 days 00:00:00.000221952,0 days 00:00:00.000303872,0 days 00:00:00.000709888,0 days 00:00:00.000854016,0 days 00:00:00.002191104,0 days 00:00:00.000310016,0 days 00:00:00.000302336,0 days 00:00:00.000301824,0 days 00:00:00.000222208,0 days 00:00:00.000571136
1134126,2016-11-03 18:09:03.999524096,2047,8,197,15,236,2,246,81,251,73,T,0 days 00:00:00.000858112,0 days 00:00:00.000221952,0 days 00:00:00.000221952,0 days 00:00:00.000303872,0 days 00:00:00.000709888,0 days 00:00:00.000854016,0 days 00:00:00.002191104,0 days 00:00:00.000310016,0 days 00:00:00.000302336,0 days 00:00:00.000301824,0 days 00:00:00.000222208


In [ ]:
fuzzy.to_csv('/content/drive/Shareddrives/빅응보 플젝/Preprocessing_Fuzzy.csv')